In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `bootpython && heroku run --app guarded-everglades-89687 make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run --app guarded-everglades-89687 make importdb && make pipeline`
# (6) Remember to delete contents of ~/Documents/links.txt
links = pd.read_csv('data/export.csv')
links

/var/folders/7f/6ty6wfcd1hs3d1wz9b1c4gsw0000gn/T/ipykernel_9199/1024356683.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108800,173479,https://aisafetychina.substack.com/p/ai-safety...,AI Safety in China #7,\N,\N,2024-01-26 20:06:18.921078+00,2024-01-26 20:06:18.921078+00,NaN,\N,AIChina,44.0,0.0,\N
108801,173480,https://aisafetychina.substack.com/p/concordia...,Concordia AI: 2023 Annual Review,\N,\N,2024-01-26 20:06:18.922026+00,2024-01-26 20:06:18.922026+00,NaN,\N,AIChina,62.0,0.0,\N
108802,173481,https://www.askamanager.org/2024/01/open-threa...,"open thread – January 26-27\, 2024",\N,\N,2024-01-26 20:06:18.922972+00,2024-01-26 20:06:18.922972+00,NaN,\N,AskManager,44.0,0.0,\N
108803,173482,https://aisafetychina.substack.com/p/new-150-p...,New 150+ Page Report on State of AI Safety in ...,\N,\N,2024-01-26 20:06:18.923905+00,2024-01-26 20:06:18.923905+00,NaN,\N,AIChina,12.0,0.0,\N


In [2]:
links['added'].max()

'2024-01-26 20:06:18.924834+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
108762    173440
108763    173441
108764    173442
108765    173443
108766    173444
Name: id, Length: 15878, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred


In [5]:
links[links['url'].duplicated()]['id']

Series([], Name: id, dtype: int64)

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/107
...11/107
...21/107
...31/107
...41/107
...51/107
...61/107
...71/107
...81/107
...91/107
...101/107
...107/107


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108907,173586,https://twitter.com/mrgunn/status/175096100570...,"@mrgunn ⏸️ on X: ""It cannot be concluded that ...",\N,\N,2024-01-27 12:51:46.433603,2024-01-27 12:51:46.433608,\N,\N,Custom,50.0,0.0,\N
108908,173587,https://www.theinformation.com/articles/the-po...,The Power and The Powder: Inside Reed Hastings...,\N,\N,2024-01-27 12:51:46.439532,2024-01-27 12:51:46.439538,\N,\N,Custom,97.0,0.0,\N
108909,173588,https://www.wikiwand.com/en/2010_flash_crash,2010 Flash Crash,\N,\N,2024-01-27 12:51:46.445553,2024-01-27 12:51:46.445558,\N,\N,Custom,94.0,0.0,\N
108910,173589,https://twitter.com/lizcooper28/status/1750956...,twitter.com/lizcooper28/status/175095648442115...,\N,\N,2024-01-27 12:51:46.451334,2024-01-27 12:51:46.451339,\N,\N,Custom,13.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108907,173586,https://twitter.com/mrgunn/status/175096100570...,"@mrgunn ⏸️ on X: ""It cannot be concluded that ...",\N,\N,2024-01-27 12:51:46.433603,2024-01-27 12:51:46.433608,\N,\N,Custom,50.0,0.0,\N
108908,173587,https://www.theinformation.com/articles/the-po...,The Power and The Powder: Inside Reed Hastings...,\N,\N,2024-01-27 12:51:46.439532,2024-01-27 12:51:46.439538,\N,\N,Custom,97.0,0.0,\N
108909,173588,https://www.wikiwand.com/en/2010_flash_crash,2010 Flash Crash,\N,\N,2024-01-27 12:51:46.445553,2024-01-27 12:51:46.445558,\N,\N,Custom,94.0,0.0,\N
108910,173589,https://twitter.com/lizcooper28/status/1750956...,twitter.com/lizcooper28/status/175095648442115...,\N,\N,2024-01-27 12:51:46.451334,2024-01-27 12:51:46.451339,\N,\N,Custom,13.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108907,173586,https://twitter.com/mrgunn/status/175096100570...,"@mrgunn ⏸️ on X: ""It cannot be concluded that ...",\N,\N,2024-01-27 12:51:46.433603,2024-01-27 12:51:46.433608,\N,\N,Custom,50,0,\N
108908,173587,https://www.theinformation.com/articles/the-po...,The Power and The Powder: Inside Reed Hastings...,\N,\N,2024-01-27 12:51:46.439532,2024-01-27 12:51:46.439538,\N,\N,Custom,97,0,\N
108909,173588,https://www.wikiwand.com/en/2010_flash_crash,2010 Flash Crash,\N,\N,2024-01-27 12:51:46.445553,2024-01-27 12:51:46.445558,\N,\N,Custom,94,0,\N
108910,173589,https://twitter.com/lizcooper28/status/1750956...,twitter.com/lizcooper28/status/175095648442115...,\N,\N,2024-01-27 12:51:46.451334,2024-01-27 12:51:46.451339,\N,\N,Custom,13,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)